# Data exploration of Yahoo Finance data
RAD for feature engineering pipeline.

In [1]:
import pandas
import yfinance as yf
from pprint import pprint
import pandas as pd
import datetime
from random import randint

In [2]:
# If yf.Ticker.info is not working running the command below solves it
# pip install yfinance --upgrade --no-cache-dir
# How to integrate this in the pipeline? Requirements?


test_symbol = 'AAPL'

stock = yf.Ticker(test_symbol)

In [3]:
def getEarnings(stock):
    return stock.earnings_history

earnings = stock.earnings_history

In [4]:
def parseMonth(month):
    """
    Function to parse the string of a month from the earnings date
    :param month: string with the given month
    :return: the number corresponding to the month in datetime format
    """
    if month == 'Jan':
        return 1
    elif month == 'Feb':
        return 2
    elif month == 'Mar':
        return 3
    elif month == 'Apr':
        return 4
    elif month == 'May':
        return 5
    elif month == 'Jun':
        return 6
    elif month == 'Jul':
        return 7
    elif month == 'Aug':
        return 8
    elif month == 'Sep':
        return 9
    elif month == 'Oct':
        return 10
    elif month == 'Nov':
        return 11
    else:
        return 12

In [5]:
def convertEarningsDate(df):
    """
    Function to convert the earnings date format to datetime
    :param df: dataframe containing the earnings history
    :return: dataframe updated with the parsed datetime as date
    """
    test = [None]*len(df)
    for i in range(len(df)):
        temp = df.loc[i, "Earnings Date"].split(', ')
        temp = [x.strip() for x in temp]
        temp = (temp[0] + ' ' + temp[1]).split(' ')
        test[i] = [temp[2], parseMonth(temp[0]), temp[1]]
    dt = pd.DataFrame(test, columns=['year', 'month', 'day'])
    df['Earnings Date'] = pd.to_datetime(dt)
    return df

In [6]:
# earnings = convertEarningsDate(earnings)

In [7]:
def getHistory(symbol, stock, period='30d', interval='30m'):
    """
    Function to retrieve the price history of the stock and parse its date
    :param stock: yfinance ticker object
    :param period: the period over which data should be collected
    :param interval: the interval for data points
    :return: history dataframe with additional columns
    """
    hist = stock.history(period = period, interval = interval)
    hist['company'] = symbol
    hist['date'] = hist.index.date
    hist['date'] = pd.to_datetime(hist['date'])
    hist['Diff'] = hist['Close'] - hist['Open']
    return hist

In [8]:
def getRelEarnings(e_df, hist_df):
    """
    Finds the earnings data which is relevant for the given history time frame
    :param e_df: earnings dataframe
    :param hist_df: history dataframe
    :return: relevant dates dataframe
    """
    minmax = hist_df['date'].agg(['min', 'max'])
    last_er_idx = e_df[e_df['Earnings Date'] <= minmax['min']].index[0]
    first_er_idx = e_df[e_df['Earnings Date'] <= minmax['max']].index[0]
    relevant_earnings = e_df[first_er_idx:last_er_idx+1].reset_index(drop=True)
    return relevant_earnings

def fillEarnings(current, hist_df, idx_in):
    """
    Function to fill the earnings columns into the history
    :param current: dataframe holding the earnings data for the selected indices
    :param hist_df: history dataframe
    :param idx_in: relevant indices on the history dataframe to fill the earnings data for
    :return: history dataframe with the earnings added for the given indices
    """
    hist_df.loc[idx_in, 'EPS Estimate'] = current['EPS Estimate']
    hist_df.loc[idx_in, 'Reported EPS'] = current['Reported EPS']
    hist_df.loc[idx_in, 'Offset'] = current['Surprise(%)']
    hist_df.loc[idx_in, 'Earnings'] = current['Earnings Date']
    return hist_df


def getHistWithEarnings(relevant_earnings, hist_df):
    """
    Function to add the corresponding earnings data to the days for which the data was known.
    :param relevant_earnings: The earnings columns which are relevant for the given history time frame
    :param hist_df: The history dataframe
    :return: History with added columns for each of the relevant earnings
    """
    for idx in reversed(relevant_earnings.index):
        if idx>0:
            current = relevant_earnings.iloc[idx]
            next = relevant_earnings.iloc[idx-1]
            idx_in = hist_df[(hist_df['date'] >= current['Earnings Date']) &
                             (hist_df['date'] < next['Earnings Date'])].index
            hist_df = fillEarnings(current, hist_df, idx_in)
        else:
            current = relevant_earnings.iloc[idx]
            idx_in = hist_df[(hist_df['date'] >= current['Earnings Date'])].index
            hist_df = fillEarnings(current, hist_df, idx_in)
    return hist_df

def dropIrrelevant(hist_df: pd.DataFrame):
    labels = ['']
    return hist_df.drop(labels, axis=1)



In [13]:
def stockToCSV(symbol, period='30d', interval='30m'):
    """
    Functions to write stock information to CSV
    :param symbol: ticker symbol of the company
    :param period: over what time period the history data should be taken
    :param interval: how often a sample is taken over the period
    :return:
    """
    stock = yf.Ticker(symbol)
    earnings = getEarnings(stock)
    earnings = convertEarningsDate(earnings)
    hist = getHistory(symbol, stock, period=period, interval=interval)
    rel_earnings = getRelEarnings(earnings, hist)
    hist = getHistWithEarnings(rel_earnings, hist)
    hist.to_csv('./data/' + symbol)

def getStocks():
    """
    Get all stock symbols listed on Nasdaq
    :return: list with symbols
    """
    df = pd.read_table('http://www.nasdaqtrader.com/dynamic/symdir/nasdaqlisted.txt')
    # df = pd.read_table('tickers.txt')
    symbols = [None]*len(df)
    for idx, line in df.iterrows():
        symbols[idx] = line[0].split('|')[0]
    return symbols

def selectStocks(stocks, n = 10):
    """
    Select n random stocks from the list
    :param stocks: list of company symbols
    :param n: number of companies to select
    :return: selected symbols
    """
    idxs = []
    selected = [None]*n
    counter = 0
    while len(idxs) < n:
        temp_int = randint(0, len(stocks))
        if temp_int not in idxs:
            idxs.append(temp_int)
            selected[counter] = stocks[temp_int]
            counter += 1
    return selected


In [14]:
stockToCSV(test_symbol)

In [17]:
getHistory('AAPL', yf.Ticker('AAPL'), '60d', '30m')

,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,date,Diff
Datetime,,,,,,,,,,
2022-09-30 09:30:00-04:00,141.680801,142.630005,140.610001,142.119995,17359317,0.0,0,AAPL,2022-09-30,0.439194
2022-09-30 10:00:00-04:00,142.119995,142.529999,141.369995,142.220001,7632656,0.0,0,AAPL,2022-09-30,0.100006
2022-09-30 10:30:00-04:00,142.220001,142.889999,141.854996,142.270096,7062240,0.0,0,AAPL,2022-09-30,0.050095
2022-09-30 11:00:00-04:00,142.289993,143.100006,141.949997,142.618805,7083136,0.0,0,AAPL,2022-09-30,0.328812
2022-09-30 11:30:00-04:00,142.620300,142.740005,141.024994,141.320007,5944641,0.0,0,AAPL,2022-09-30,-1.300293
...,...,...,...,...,...,...,...,...,...,...
2022-12-23 10:30:00-05:00,131.395004,132.414993,130.750000,130.870102,5766852,0.0,0,AAPL,2022-12-23,-0.524902
2022-12-23 11:00:00-05:00,130.880005,131.380005,130.253098,131.008499,5149930,0.0,0,AAPL,2022-12-23,0.128494
2022-12-23 11:30:00-05:00,131.009995,131.690002,130.729996,130.875000,3504501,0.0,0,AAPL,2022-12-23,-0.134995


In [18]:
pd.read_csv('data/AAPL')

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,date,Diff,EPS Estimate,Reported EPS,Offset,Earnings
0,2022-11-11 09:30:00-05:00,145.625000,146.470001,144.369995,145.440002,13745548,0,0,AAPL,2022-11-11,-0.184998,1.27,1.29,1.57,2022-10-27
1,2022-11-11 10:00:00-05:00,145.419998,148.022507,144.750107,147.600006,9327287,0,0,AAPL,2022-11-11,2.180008,1.27,1.29,1.57,2022-10-27
2,2022-11-11 10:30:00-05:00,147.600006,148.570007,147.070007,148.115005,6286433,0,0,AAPL,2022-11-11,0.514999,1.27,1.29,1.57,2022-10-27
3,2022-11-11 11:00:00-05:00,148.119995,148.750000,146.699997,146.699997,6136245,0,0,AAPL,2022-11-11,-1.419998,1.27,1.29,1.57,2022-10-27
4,2022-11-11 11:30:00-05:00,146.714996,147.649994,146.429993,147.380005,5210977,0,0,AAPL,2022-11-11,0.665009,1.27,1.29,1.57,2022-10-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2022-12-23 10:00:00-05:00,130.020004,131.460007,129.660507,131.389999,7954458,0,0,AAPL,2022-12-23,1.369995,1.27,1.29,1.57,2022-10-27
374,2022-12-23 10:30:00-05:00,131.395004,132.414993,130.750000,130.870102,5766852,0,0,AAPL,2022-12-23,-0.524902,1.27,1.29,1.57,2022-10-27
375,2022-12-23 11:00:00-05:00,130.880005,131.380005,130.253098,131.008499,5149930,0,0,AAPL,2022-12-23,0.128494,1.27,1.29,1.57,2022-10-27
376,2022-12-23 11:30:00-05:00,131.009995,131.690002,130.729996,130.875000,3504501,0,0,AAPL,2022-12-23,-0.134995,1.27,1.29,1.57,2022-10-27
